In [1]:
!pip install pandas openpyxl xlrd


Defaulting to user installation because normal site-packages is not writeable


In [32]:
import pandas as pd

df = pd.read_excel(r'C:\Users\Admin\Downloads\JOB WORK INVOICE SHM023 DT19062025 SHIM JOB WORK-LGD.xlsx', sheet_name='PO Copy.',skiprows=2)


selected_columns = ['Vendor Style#', 'SIZE', 'Ord Qty', 
                    'KT/Col', 'NY PO#','Spl. Remark','Min / Max','Stamp Instr','Cust','NY Style #','LGD QUALITY','Mined/LGD','Indigo Order#']

# Explicit copy
df_selected = df[selected_columns].copy()

# Rename columns
df_selected.rename(columns={
    'Vendor Style#': 'StyleCode',
    'SIZE': 'ItemSize',
    'Ord Qty': 'OrderQty',
    'NY PO#': 'ItemPoNo',
    'Spl. Remark': 'CustomerProductionInstruction',
    'Stamp Instr' : 'StampInstruction',
    'Cust' : 'OrderGroup',
    'NY Style #' : 'SKUNo'
}, inplace=True)

df_selected.insert(
    df_selected.columns.get_loc('StyleCode'),
    'SrNo',
    range(1, len(df_selected) + 1)  # incremental numbers starting from 1
)

# Function to map KT/Col to Metal code
def metal_code(kt_col):
    if pd.isna(kt_col):
        return ''
    
    kt_col = str(kt_col).upper().strip()
    
    # Platinum
    if '950PT' in kt_col:
        return 'PT950'
    
    # Karat gold
    if 'KT' in kt_col:
        if '/' in kt_col:
            karat, color = kt_col.split('/')
            karat = karat.replace('KT','')
            if color == 'W':
                return f'G{karat}W'
            elif color == 'Y':
                return f'G{karat}Y'
            elif color == 'P':
                return f'G{karat}P'
            else:
                return f'G{karat}X'  # unknown color
        else:
            karat = kt_col.replace('KT','')
            return f'G{karat}X'
    
    return 'UNKNOWN'

# Insert Metal column after OrderQty
df_selected.insert(
    df_selected.columns.get_loc('OrderQty') + 1,
    'Metal',
    df_selected['KT/Col'].apply(metal_code)
)

# Function to extract tone from KT/Col
def extract_tone(kt_col):
    if pd.isna(kt_col):
        return ''
    kt_col = str(kt_col).upper().strip()
    if '/' in kt_col:
        return kt_col.split('/')[-1]  # take last part after "/"
    return ''  # if no slash, return blank

# Insert Tone column after Metal
df_selected.insert(
    df_selected.columns.get_loc('Metal') + 1,
    'Tone',
    df_selected['KT/Col'].apply(extract_tone)
)

# Create DesignProductionInstruction by combining CustomerProductionInstruction and Min / Max
def design_instruction(row):
    cpi = '' if pd.isna(row['CustomerProductionInstruction']) else str(row['CustomerProductionInstruction']).strip()
    minmax = '' if pd.isna(row['Min / Max']) else str(row['Min / Max']).strip()
    
    if cpi and minmax:
        return f"{cpi}, {minmax}"
    elif cpi:
        return cpi
    elif minmax:
        return minmax
    else:
        return ''

# Insert DesignProductionInstruction column after Tone
df_selected.insert(
    df_selected.columns.get_loc('Tone') + 1,
    'DesignProductionInstruction',
    df_selected.apply(design_instruction, axis=1)
)

# Function to create SpecialRemarks
def create_special_remarks(row):
    parts = []
    # Add each part if it exists
    if not pd.isna(row['OrderGroup']):
        parts.append(str(row['OrderGroup']).strip())
    if not pd.isna(row['StyleCode']):
        parts.append(str(row['StyleCode']).strip())
    if not pd.isna(row['KT/Col']):
        parts.append(str(row['KT/Col']).strip())
    if not pd.isna(row['LGD QUALITY']):
        parts.append(str(row['LGD QUALITY']).strip())
    if not pd.isna(row['Mined/LGD']):
        parts.append(str(row['Mined/LGD']).strip())
    if not pd.isna(row['Indigo Order#']):
        parts.append(str(row['Indigo Order#']).strip())
    
    return ', '.join(parts)

# Insert SpecialRemarks column after CustomerProductionInstruction
df_selected.insert(
    df_selected.columns.get_loc('CustomerProductionInstruction') + 1,
    'SpecialRemarks',
    df_selected.apply(create_special_remarks, axis=1)
)

df_selected.insert(
    df_selected.columns.get_loc('OrderQty') + 1,
    'OrderItemPcs',
    ''  # blank by default
)

# Move ItemPoNo column after Tone
item_po_no = df_selected.pop('ItemPoNo')
df_selected.insert(df_selected.columns.get_loc('Tone') + 1, 'ItemPoNo', item_po_no)

# List of new columns to add after ItemPoNo
new_cols = ['ItemRefNo', 'StockType', 'MakeType']

# Insert each new column as blank after ItemPoNo
pos = df_selected.columns.get_loc('ItemPoNo') + 1
for col in new_cols:
    df_selected.insert(pos, col, '')
    pos += 1
    
# Move DesignProductionInstruction after SpecialRemarks
design_col = df_selected.pop('DesignProductionInstruction')
df_selected.insert(df_selected.columns.get_loc('SpecialRemarks') + 1, 'DesignProductionInstruction', design_col)

# List of additional new columns to add after SKUNo
additional_cols = [
    'Basestoneminwt', 'Basestonemaxwt', 'Basemetalminwt', 'Basemetalmaxwt',
    'Productiondeliverydate', 'Expecteddeliverydate', 'SetPrice', 'StoneQuality'
]

# Insert each new column as blank after SKUNo
pos = df_selected.columns.get_loc('SKUNo') + 1
for col in additional_cols:
    df_selected.insert(pos, col, '')
    pos += 1

df_selected.drop(columns=['Mined/LGD','Indigo Order#','KT/Col','Min / Max','LGD QUALITY'], inplace=True) 
#df_selected.to_csv(r'C:\Users\Admin\Desktop\SHM023 DT19062025_PO_Cleaned.csv', index=False)
df_selected.head()

,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo,ItemRefNo,StockType,...,OrderGroup,SKUNo,Basestoneminwt,Basestonemaxwt,Basemetalminwt,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate,SetPrice,StoneQuality
0,1,VR4178SEB1,7.00,7,,G14W,W,208538,,,...,REED,B1155BPC1.00RJ,,,,,,,,
1,2,XR3843L,7.00,1,,G10 X,W,408815,,,...,HZ,LGF4060-10W-18A,,,,,,,,
2,3,XR5357ME,10.50,1,,G14 X,Y,408817,,,...,RH,B1342AE2.0RH,,,,,,,,
3,4,WR4335JG,10.00,1,,G14 X,YW,409510-BN,,,...,HZ,LGG4010-14YW-96A,,,,,,,,
4,5,SVBRC24367XX,6.75,1,,UNKNOWN,W,208704,,,...,IDI,-,,,,,,,,


In [35]:
#Final Code
import pandas as pd

df = pd.read_excel(r'C:\Users\Admin\Downloads\Po# Mounting-1.xlsx', sheet_name='PO Copy.',skiprows=1)


selected_columns = ['Vendor Style#', 'SIZE', 'Ord Qty', 
                    'KT/Col', 'NY PO#','Spl. Remark','Min / Max','Stamp Instr','Cust','NY Style #','LGD QUALITY','Mined/LGD','Indigo Order#']

# Explicit copy
df_selected = df[selected_columns].copy()

# Rename columns
df_selected.rename(columns={
    'Vendor Style#': 'StyleCode',
    'SIZE': 'ItemSize',
    'Ord Qty': 'OrderQty',
    'NY PO#': 'ItemPoNo',
    'Spl. Remark': 'CustomerProductionInstruction',
    'Stamp Instr' : 'StampInstruction',
    'Cust' : 'OrderGroup',
    'NY Style #' : 'SKUNo'
}, inplace=True)

df_selected.insert(
    df_selected.columns.get_loc('StyleCode'),
    'SrNo',
    range(1, len(df_selected) + 1)  # incremental numbers starting from 1
)

# Function to map KT/Col to Metal code
def metal_code(kt_col):
    if pd.isna(kt_col):
        return ''
    
    kt_col = str(kt_col).upper().strip().replace(' ', '')  # remove spaces
    
    # Platinum
    if '950PT' in kt_col or 'PT950' in kt_col:
        return 'PT950'
    
    # Karat gold
    if 'KT' in kt_col:
        if '/' in kt_col:
            karat, color = kt_col.split('/')
            karat = karat.replace('KT','')
            color = color.strip()
            return f'G{karat}{color}'  # e.g., G14Y, G14YW
        else:
            karat = kt_col.replace('KT','')
            return f'G{karat}X'  # unknown color
    return 'UNKNOWN'

# Insert Metal column after OrderQty
df_selected.insert(
    df_selected.columns.get_loc('OrderQty') + 1,
    'Metal',
    df_selected['KT/Col'].apply(metal_code)
)

# Function to extract tone from KT/Col
def extract_tone(kt_col):
    if pd.isna(kt_col):
        return ''
    kt_col = str(kt_col).upper().strip()
    if '/' in kt_col:
        return kt_col.split('/')[-1]  # take last part after "/"
    return ''  # if no slash, return blank

# Insert Tone column after Metal
df_selected.insert(
    df_selected.columns.get_loc('Metal') + 1,
    'Tone',
    df_selected['KT/Col'].apply(extract_tone)
)

# Create DesignProductionInstruction by combining CustomerProductionInstruction and Min / Max
def design_instruction(row):
    cpi = '' if pd.isna(row['CustomerProductionInstruction']) else str(row['CustomerProductionInstruction']).strip()
    minmax = '' if pd.isna(row['Min / Max']) else str(row['Min / Max']).strip()
    
    if cpi and minmax:
        return f"{cpi}, {minmax}"
    elif cpi:
        return cpi
    elif minmax:
        return minmax
    else:
        return ''

# Insert DesignProductionInstruction column after Tone
df_selected.insert(
    df_selected.columns.get_loc('Tone') + 1,
    'DesignProductionInstruction',
    df_selected.apply(design_instruction, axis=1)
)

# Function to create SpecialRemarks
def create_special_remarks(row):
    parts = []
    # Add each part if it exists
    if not pd.isna(row['OrderGroup']):
        parts.append(str(row['OrderGroup']).strip())
    if not pd.isna(row['StyleCode']):
        parts.append(str(row['StyleCode']).strip())
    if not pd.isna(row['KT/Col']):
        parts.append(str(row['KT/Col']).strip())
    if not pd.isna(row['LGD QUALITY']):
        parts.append(str(row['LGD QUALITY']).strip())
    if not pd.isna(row['Mined/LGD']):
        parts.append(str(row['Mined/LGD']).strip())
    if not pd.isna(row['Indigo Order#']):
        parts.append(str(row['Indigo Order#']).strip())
    
    return ', '.join(parts)

# Insert SpecialRemarks column after CustomerProductionInstruction
df_selected.insert(
    df_selected.columns.get_loc('CustomerProductionInstruction') + 1,
    'SpecialRemarks',
    df_selected.apply(create_special_remarks, axis=1)
)

df_selected.insert(
    df_selected.columns.get_loc('OrderQty') + 1,
    'OrderItemPcs',
    ''  # blank by default
)

# Move ItemPoNo column after Tone
item_po_no = df_selected.pop('ItemPoNo')
df_selected.insert(df_selected.columns.get_loc('Tone') + 1, 'ItemPoNo', item_po_no)

# List of new columns to add after ItemPoNo
new_cols = ['ItemRefNo', 'StockType', 'MakeType']

# Insert each new column as blank after ItemPoNo
pos = df_selected.columns.get_loc('ItemPoNo') + 1
for col in new_cols:
    df_selected.insert(pos, col, '')
    pos += 1
    
# Move DesignProductionInstruction after SpecialRemarks
design_col = df_selected.pop('DesignProductionInstruction')
df_selected.insert(df_selected.columns.get_loc('SpecialRemarks') + 1, 'DesignProductionInstruction', design_col)

# List of additional new columns to add after SKUNo
additional_cols = [
    'Basestoneminwt', 'Basestonemaxwt', 'Basemetalminwt', 'Basemetalmaxwt',
    'Productiondeliverydate', 'Expecteddeliverydate', 'SetPrice', 'StoneQuality'
]

# Insert each new column as blank after SKUNo
pos = df_selected.columns.get_loc('SKUNo') + 1
for col in additional_cols:
    df_selected.insert(pos, col, '')
    pos += 1

df_selected.drop(columns=['Mined/LGD','Indigo Order#','KT/Col','Min / Max','LGD QUALITY'], inplace=True) 
df_selected.to_csv(r'C:\Users\Admin\Desktop\p_PO_Cleaned.csv', index=False)
df_selected.head()

,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo,ItemRefNo,StockType,...,OrderGroup,SKUNo,Basestoneminwt,Basestonemaxwt,Basemetalminwt,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate,SetPrice,StoneQuality
0,1,IA4055-PTW-95A,-,10,,PT950,W,Mounting-1,,,...,INDNY-BE,IA4055-PTW-95A,,,,,,,,
1,2,IA4057-PTW-95A,-,7,,PT950,W,Mounting-1,,,...,INDNY-BE,IA4057-PTW-95A,,,,,,,,
